# 1. Activity Magic

The Activity Magic is a method of giving quizzes, polls, etc. to a group of users on a jupyterhub server. 

In the current implementation, it acts like a Classroom Response System (also called "clickers", [audience response system](https://en.wikipedia.org/wiki/Audience_response), personal response systems, student response system, or electronic response system).

The main pedagogical use is for implementing instantaneous anonymous polling for gaging student understanding.

# Table of Contents
* [1. Activity Magic](#1.-Activity-Magic)
	* [1.1 Requirements](#1.1-Requirements)
		* [1.1.1 Install tl;dr](#1.1.1-Install-tl;dr)
	* [1.2 Workflow](#1.2-Workflow)
	* [1.3 Demonstration](#1.3-Demonstration)
	* [1.4 JSON Details](#1.4-JSON-Details)
		* [1.4.1 Top level](#1.4.1-Top-level)
		* [1.4.2 Activity types](#1.4.2-Activity-types)
			* [1.4.2.1 poll](#1.4.2.1-poll)
	* [1.5 Results file format](#1.5-Results-file-format)


## 1.1 Requirements

To use, you will need:

* [jupyterhub](https://github.com/jupyter/jupyterhub) - for multi-user notebook server
* [metakernel](https://github.com/Calysto/metakernel/) - for the [activity magic](https://github.com/Calysto/metakernel/blob/master/metakernel/magics/activity_magic.py)
* [calysto](https://github.com/Calysto/calysto) - for barcharts

### 1.1.1 Install tl;dr

1. Shell: pip install metakernel
2. Shell: pip install calysto
3. Notebook: from metakernel import register_ipython_magics
4. Notebook: register_ipython_magics()

## 1.2 Workflow

Activity magic works as follows:

1. instructor creates a poll using the %%activity cell magic defining a poll via JSON
2. poll appears for verification
3. students use the same file, but via a line magic
4. everyone should make their choice
5. instructor shows bar chart of all choices
6. everyone clicks on "Next" to proceed to the next question

## 1.3 Demonstration

Here is the docs on %%activity magic:

In [1]:
%%activity?

```
Docstring:
%%activity FILENAME - make an activity from
  a JSON structure

This magic will construct a Python file from the cell's
content, a JSON structure.

Example:
    %%activity /home/teacher/activity1
    {"activity": "poll",
     "instructors": ["teacher01"],
     "items": [{"id": "...",
                "type": "multiple choice",
                "question": "...",
                "options": ["...", ...]
               }, ...]
    }

In this example, users will load
/home/teacher/activity1
File:      /usr/lib/python3.4/site-packages/metakernel-0.10.5-py3.4.egg/metakernel/magics/activity_magic.py
```

Here is an actual poll with three questions:

In [4]:
%%activity /home/dblank/activity1

{"activity": "poll",
 "instructors": ["dblank"],
 "items": [
      {"id": "1", 
       "question":  """Which of the following will print "Hello" 5 times without errors?""", 
       "type": "multiple choice",
       "options": [
"""<pre style="width: 600px">
for (int i=0; i < 5; i++) {
    println("Hello");
}
</pre>""",
"""<pre style="width: 600px">
println("Hello");
println("Hello");
println("Hello");
println("Hello");
println("Hello");
</pre>""",
"""<pre style="width: 600px">
int i = 0;
while (i < 5) {
    println("Hello");
    i++;
}
</pre>""",
        "All of the above",
        "None of the above",
    ]},
        
    {"id": "2", 
     "question":    """Which of the following is a function definition?""", 
     "type": "multiple choice",
     "options":    [
"""<pre style="width: 600px">
void draw() {
}
</pre>""",
"""<pre style="width: 600px">
rect(10, 10, 50, 50);
</pre>""",
"""<pre style="width: 600px">
drawBunny(mouseX, mouseY, 50, 100);
</pre>""",
        "All of the above",
        "None of the above",
        ]},
      {"id": "3", 
       "question":     """Which of the following has parameters?""", 
       "type": "multiple choice",
       "options":     [
"""<pre style="width: 600px">
void draw() {
}
</pre>""",
"""<pre style="width: 600px">
void drawRect(float x, float y, float w, float h) {
    rect(x, y, w, h);
}
</pre>""",
"""<pre style="width: 600px">
void drawBunny() {
    rect(10, 20, 100, 150);
}
</pre>""",
        "All of the above",
        "None of the above",
        ]},
     {"id": "4", 
      "question":  """Which of the following is a function call?""", 
      "type": "multiple choice",
      "options": [
"""<pre style="width: 600px">
void draw() {
}
</pre>""",
"""<pre style="width: 600px">
rect(10, 10, 50, 50);
</pre>""",
"""<pre style="width: 600px">
drawBunny(mouseX, mouseY, 50, 100);
</pre>""",
        "2 and 3",
        "None of the above",
        ]},
    ]
}




When executed in a live cell, it shows the following:

<img src="images/activity.png">

Instructors in the list will be able to click on the "Results" button to show the barchart of choices.

Students enter:

In [2]:
%activity /home/dblank/activity1

and are ready for discussing issues.

## 1.4 JSON Details

The activities are stored in a JSON-like format as follows.

### 1.4.1 Top level

```python
{
    "activity": "poll",
    "instructors": ["userid", ...],
    "items": [...],
    "results_filename": "/home/user/results.txt"
}
```

* **activity** is an activity type
* **instructors** is optional, and lists those user id's for which the "Results" button will appear.
* **item** is a list of items specific to this activity type
* **results_filename** is optional; if unspecified, it will be the name of the activity file + ".results". The system will automatically create it and make it writable by everyone

### 1.4.2 Activity types

#### 1.4.2.1 poll

Poll has the following structure for an item:

```python
{
    "id": "1",
    "type": "multiple choice",
    "question": "The actual question?",
    "options": ["choice 1", "choice 2", ...]
}
```

Currently, all of the poll items are strings that will be rendered as HTML. Future plans include adding all widget types (LaTeX, text, etc.) The options are currently limited to 5 choices or less. Also, choices are listed as 1 through 5.

* **id** is a unique string identification that you create
* **type** is a poll activity type
* **question** is the question to ask
* **options** is a list of the multiple choices

## 1.5 Results file format

```python
question id::user id::datetime::choice
```

The barchart is rendered by [calysto's graphics](https://github.com/Calysto/calysto).

Try it out! Feedback welcome!